In [1]:
os.chdir('/home/lgorman/Desktop/legumeCHOICE/data-processing') 
import pandas as pd 
import json
import pymongo
import os
import shutil
from dataProcessing import agroEcoData, contextData, legumeData, pairwiseData, participatoryMatrixData

In [2]:
client = pymongo.MongoClient("localhost", 27017)
db = client["legume-choice"]
print(db.name)
# Extracting the projects data
projectsData =  db["projects"]
# Finding the number of projects
print(projectsData.count_documents({}))


# Extacting all projects
allProjects = []
for project in projectsData.find():
        allProjects.append(project)
projectIDs = [project["projectID"] for project in allProjects]
# Example querying from the dataset
#allProjects[0]["rawdata"]["agroEcoData"]

legume-choice
4


In [3]:
homepath = os.getenv('HOME')
os.chdir(homepath) 

private_data_path = os.path.join(homepath, "Desktop", "PrivateLegumeCHOICE")

public_data_path = os.path.join(homepath, "Desktop", "PublicLegumeCHOICE")
public_data_individual_path = os.path.join(homepath, "Desktop", "PublicLegumeCHOICE", "individualProjects")
public_data_assembled_path = os.path.join(homepath, "Desktop", "PublicLegumeCHOICE", "aggregatedProjects")

if os.path.exists(private_data_path) is False:
    os.mkdir(private_data_path)

if os.path.exists(public_data_path) is False:
    os.mkdir(private_data_path)

# Individual data. Delete folder then recreate for all new calculations
if os.path.exists(public_data_individual_path) is True:
    shutil.rmtree(public_data_individual_path)
    os.mkdir(public_data_individual_path)


if os.path.exists(public_data_individual_path) is False:
    os.mkdir(public_data_individual_path)

# Aggregated data. Delete folder then recreate for all new calculations
if os.path.exists(public_data_assembled_path) is True:
    shutil.rmtree(public_data_assembled_path)
    os.mkdir(public_data_assembled_path)

if os.path.exists(public_data_assembled_path) is False:
    os.mkdir(public_data_assembled_path)

# Create an empty project directory for each individual project
for projectID in projectIDs:
    new_path = os.path.join(public_data_individual_path, projectID)
    os.mkdir(new_path)

## Single Project Information

In [4]:
project = allProjects[0]
for project in allProjects:

    publicfolder = os.path.join(public_data_individual_path, project["projectID"])
    privatefile =  os.path.join(private_data_path, project["projectID"]+".json")
# AgroEco information is only one row, so have to transform
    agroEcoSingleProj = pd.DataFrame(columns=list(agroEcoData.ExtractProjectAgroEcoData(project).keys()))
    agroEcoSingleProj.loc[0]=agroEcoData.ExtractProjectAgroEcoData(project).values()


    contextSingleProj = pd.DataFrame(contextData.projectContextScores(project=project))
    pairwiseSelectionsSingleProj = pairwiseData.pairWiseSelectionsProject(project=project)
    pairwideSummarySingleProj = pairwiseData.pairWiseSummaryScores(project=project)
    participatoryMatrixSingleProj = participatoryMatrixData.participatoryMatrixScoresProject(project=project)
    legumeDataSingleProj = legumeData.extractAllLegumeData(project=project)

    agroEcoSingleProj.to_csv(os.path.join(publicfolder,"agroEcoData.csv"))
    contextSingleProj.to_csv(os.path.join(publicfolder,"ContextData.csv"))
    pairwiseSelectionsSingleProj.to_csv(os.path.join(publicfolder,"PairwiseSelections.csv"))
    pairwideSummarySingleProj.to_csv(os.path.join(publicfolder,"PairwiseSummaryScores.csv"))
    participatoryMatrixSingleProj.to_csv(os.path.join(publicfolder,"ParticipatoryMatrixScores.csv"))
    legumeDataSingleProj.to_csv(os.path.join(publicfolder,"LegumeResults.csv"))

    with open(privatefile, "w") as outfile:
        JSONData = {
            "currentProject":project["rawdata"]
        }
        json.dump(JSONData,outfile, indent=4)




    
    



In [5]:



agroEcoAllProj = agroEcoData.ExtractAllAgroEcoData(allProjects)
contextAllProj = contextData.ContextScoresAllProjects(projects=allProjects)
pairwiseAllProj = pairwiseData.pairWiseSelectionsAllProjects(projects=allProjects)
pairwideSummaryAllProj = pairwiseData.pairWiseSelectionSummaryAllProjects(projects=allProjects)
participatoryMatrixAllProj = participatoryMatrixData.participatoryMatrixAllProjects(projects=allProjects)
legumeDataAllProj = legumeData.legumeScoresAllProjects(projects=allProjects)

agroEcoAllProj.to_csv(os.path.join(public_data_assembled_path,"agroEcoData.csv"))
contextAllProj.to_csv(os.path.join(public_data_assembled_path,"ContextData.csv"))
pairwiseAllProj.to_csv(os.path.join(public_data_assembled_path,"PairwiseSelections.csv"))
pairwideSummaryAllProj.to_csv(os.path.join(public_data_assembled_path,"PairwiseSummaryScores.csv"))
participatoryMatrixAllProj.to_csv(os.path.join(public_data_assembled_path,"ParticipatoryMatrixScores.csv"))
legumeDataAllProj.to_csv(os.path.join(public_data_assembled_path,"LegumeResults.csv"))





/home/lgorman/Desktop/legumeCHOICE/data-processing


NameError: name '__file__' is not defined